In [1]:
!nvidia-smi

Thu Nov 25 07:08:48 2021       
+-----------------------------------------------------------------------------+
| NVIDIA-SMI 460.67       Driver Version: 460.67       CUDA Version: 11.2     |
|-------------------------------+----------------------+----------------------+
| GPU  Name        Persistence-M| Bus-Id        Disp.A | Volatile Uncorr. ECC |
| Fan  Temp  Perf  Pwr:Usage/Cap|         Memory-Usage | GPU-Util  Compute M. |
|                               |                      |               MIG M. |
|===============================+======================+======================|
|   0  GeForce RTX 3090    Off  | 00000000:65:00.0 Off |                  N/A |
|100%   77C    P2   318W / 370W |  23632MiB / 24265MiB |     70%      Default |
|                               |                      |                  N/A |
+-------------------------------+----------------------+----------------------+
|   1  GeForce RTX 3090    Off  | 00000000:B3:00.0 Off |                  N/A |
| 68%   

In [2]:
import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import skimage.transform as st
import os
import gc
import warnings
from sklearn.metrics import classification_report, average_precision_score
from Parse_TFrecords import *
from define_model import *
from load_data import *
from utilities import *
 
print(tf.__version__)

warnings.filterwarnings("ignore")
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '2'

gpus = tf.config.list_physical_devices(device_type='GPU')
tf.config.set_visible_devices(devices=gpus[0], device_type='GPU')

2.5.0


## Model Ensembling 

In [3]:
pos = 27297
neg = 119133

total = pos+neg

weight_for_0 = (1 / neg) * (total / 2.0)
weight_for_1 = (1 / pos) * (total / 2.0)

class_weight = {0: weight_for_0, 1: weight_for_1}

print('Weight for class 0: {:.2f}'.format(weight_for_0))
print('Weight for class 1: {:.2f}'.format(weight_for_1))

Weight for class 0: 0.61
Weight for class 1: 2.68


In [4]:
BATCH_SIZE = 32

record_file_train = 'copd_train_new.tfrecords'
train_dataset = (tf.data.TFRecordDataset(
    record_file_train, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(total)
.batch(BATCH_SIZE))

record_file_val = 'copd_val_new.tfrecords'
val_dataset = (tf.data.TFRecordDataset(
    record_file_val, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_train)
.shuffle(36601)
.batch(BATCH_SIZE))

record_file_test = 'copd_test_new.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test)
.batch(BATCH_SIZE))

y_train = get_data_label('train')
y_val = get_data_label('val')
y_test = get_data_label('test')

## Train base model

In [5]:
archi = 'Dnet121'

checkpoint_filepath = 'checkpoints/Recall/checkpoint_Dnet121'
monitor_ = 'val_recall'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

model = load_model_from_pretrain(archi)

# metric = tf.keras.metrics.PrecisionAtRecall(recall=0.5, name='precision_at_recall')
metric = tf.keras.metrics.Recall(name='recall')
# metric = tf.keras.metrics.Precision(name='precision')

model.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

model.fit(train_dataset, epochs=20, shuffle=True, validation_data=val_dataset, callbacks=callback, class_weight=class_weight)

y_preds = model.predict(test_dataset)

test_CI(y_preds, y_test)
    
thresh = get_thresh(y_test, y_preds, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = get_thresh(y_test, y_preds, 'G-mean')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

thresh = 0.5
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

gc.collect()

Instructions for updating:
The `validate_indices` argument has no effect. Indices are always validated on CPU and never validated on GPU.
Epoch 1/20
4576/4576 [==============================] - 443s 92ms/step - loss: 0.6133 - recall: 0.6776 - val_loss: 0.8504 - val_recall: 0.8048
Epoch 2/20
4576/4576 [==============================] - 428s 92ms/step - loss: 0.5788 - recall: 0.7068 - val_loss: 0.6100 - val_recall: 0.6797
Epoch 3/20
4576/4576 [==============================] - 427s 92ms/step - loss: 0.5491 - recall: 0.7264 - val_loss: 0.5904 - val_recall: 0.6312
Epoch 4/20
4576/4576 [==============================] - 427s 92ms/step - loss: 0.5180 - recall: 0.7511 - val_loss: 0.5000 - val_recall: 0.4492
Sample auc mean: 0.72
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.71 - 0.72]
thresh: 0.3044091
              precision    recall  f1-score   support

           0       0.89      0.65      0.75     36808
           1       0.32      0.67      0.43      9026

  

4

## Prediction-based ensemble

In [4]:
BATCH_SIZE = 32

record_file_test = 'copd_test_new.tfrecords'
test_dataset = (tf.data.TFRecordDataset(
    record_file_test, buffer_size=BATCH_SIZE*10, compression_type=None, num_parallel_reads=32)
.map(parse_TFrecord_test)
.batch(BATCH_SIZE))

y_test = get_data_label('test')

In [18]:
checkpoint_filepath = 'checkpoints/Recall/checkpoint_Dnet121'
archi = 'Dnet121'
model1 = define_model(archi)
model1.load_weights(checkpoint_filepath)

checkpoint_filepath = 'checkpoints/Precision/checkpoint_Dnet121'
archi = 'Dnet121'
model2 = define_model(archi)
model2.load_weights(checkpoint_filepath)

y_preds_1 = model1.predict(test_dataset)
y_preds_2 = model2.predict(test_dataset)

In [19]:
y_preds_vote = (y_preds_1*0.35 + y_preds_2*0.65)
    
y_preds_vote = np.array(y_preds_vote)
    
test_CI(y_preds_vote, y_test)

print(average_precision_score(y_test, y_preds_vote, average=None))
    
thresh = get_thresh(y_test, y_preds_vote, 'Youden')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds_vote >= thresh, 1, 0)))

thresh = get_thresh(y_test, y_preds_vote, 'G-mean')
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds_vote >= thresh, 1, 0)))

thresh = 0.5
print('thresh:', thresh)
        
print(classification_report(y_test, np.where(y_preds_vote >= thresh, 1, 0)))

gc.collect()

Sample auc mean: 0.72
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.72 - 0.73]
0.3785993913465464
thresh: 0.25984934
              precision    recall  f1-score   support

           0       0.89      0.63      0.74     36808
           1       0.32      0.70      0.44      9026

    accuracy                           0.64     45834
   macro avg       0.61      0.66      0.59     45834
weighted avg       0.78      0.64      0.68     45834

thresh: 0.25984934
              precision    recall  f1-score   support

           0       0.89      0.63      0.74     36808
           1       0.32      0.70      0.44      9026

    accuracy                           0.64     45834
   macro avg       0.61      0.66      0.59     45834
weighted avg       0.78      0.64      0.68     45834

thresh: 0.5
              precision    recall  f1-score   support

           0       0.84      0.89      0.86     36808
           1       0.41      0.32      0.36      9026

    acc

0

## Model-based ensemble

In [14]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_BCE_IV3'
archi = 'IV3'
model1 = load_model_from_pretrain(archi)
model1.load_weights(checkpoint_filepath)

checkpoint_filepath = 'checkpoints/AUC/checkpoint_BCE_Dnet121'
archi = 'Dnet121'
model2 = load_model_from_pretrain(archi)
model2.load_weights(checkpoint_filepath)

model1 = tf.keras.Model(inputs=model1.input, outputs=model1.get_layer(model1.layers[-2].name).output)
model2 = tf.keras.Model(inputs=model2.input, outputs=model2.get_layer(model2.layers[-2].name).output)

features_1 = model1.predict(train_dataset)
features_2 = model2.predict(train_dataset)

features_val_1 = model1.predict(val_dataset)
features_val_2 = model2.predict(val_dataset)

In [16]:
checkpoint_filepath = 'checkpoints/AUC/checkpoint_ensemble_mlp'
monitor_ = 'val_auc'
model_checkpoint_callback = tf.keras.callbacks.ModelCheckpoint(
    filepath=checkpoint_filepath,
    save_best_only=True,
    save_weights_only=True,
    monitor=monitor_,
    mode='max')
    
callback = [tf.keras.callbacks.LearningRateScheduler(scheduler),
            tf.keras.callbacks.EarlyStopping(mode='max', patience=3, monitor=monitor_),
            model_checkpoint_callback]

ensemble_mlp = get_ensemble_mlp()

metric = tf.keras.metrics.AUC(name='auc')

ensemble_mlp.compile(loss=tf.keras.losses.BinaryCrossentropy(from_logits=True),
                 optimizer=tf.keras.optimizers.Adam(learning_rate=1e-3), metrics=metric)

ensemble_mlp.fit([features_IV3, features_D121], y_train, epochs=20, shuffle=True, validation_data=([features_val_IV3, features_val_D121],  y_val), callbacks=callback, class_weight=class_weight)

Epoch 1/20
4576/4576 [==============================] - 23s 5ms/step - loss: 0.6948 - precision_at_recall: 0.1869 - val_loss: 0.6908 - val_precision_at_recall: 0.1734
Epoch 2/20
4576/4576 [==============================] - 22s 5ms/step - loss: 0.6934 - precision_at_recall: 0.1865 - val_loss: 0.6891 - val_precision_at_recall: 0.1723
Epoch 3/20
4576/4576 [==============================] - 22s 5ms/step - loss: 0.6932 - precision_at_recall: 0.1864 - val_loss: 0.6925 - val_precision_at_recall: 0.1723
Epoch 4/20
4576/4576 [==============================] - 22s 5ms/step - loss: 0.6932 - precision_at_recall: 0.1864 - val_loss: 0.6905 - val_precision_at_recall: 0.1723


In [17]:
X_test_IV3 = InceptionV3.predict(test_dataset)
X_test_D121 = densenet121.predict(test_dataset)

y_preds = ensemble_mlp.predict([X_test_IV3, X_test_D121])

test_CI(y_preds, y_test)

thresh = get_thresh(y_test, y_preds)

print(classification_report(y_test, np.where(y_preds >= thresh, 1, 0)))
print(average_precision_score(y_test, y_preds, average=None))

gc.collect()

Sample auc mean: 0.44
Samole auc std: 0.00
Sample auc CI: 0.01
Confidence interval for AUC: [0.43 - 0.45]
              precision    recall  f1-score   support

           0       1.00      0.00      0.00     36808
           1       0.20      1.00      0.33      9026

    accuracy                           0.20     45834
   macro avg       0.60      0.50      0.16     45834
weighted avg       0.84      0.20      0.07     45834

0.17071281818144116


746060